In [2]:
#pip install nibabel


     |████████████████████████████████| 3.3 MB 3.7 MB/s eta 0:00:01
     |████████████████████████████████| 17.3 MB 26.4 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import glob
import json
import nibabel as nib
import numpy as np


# Define the base BIDS directory
bids_dir = "/projects/NHP_processed/developmental_in_modified"

for subject_dir in glob.glob(os.path.join(bids_dir, "sub-*")):
    subject_id = os.path.basename(subject_dir)

    for session_dir in glob.glob(os.path.join(subject_dir, "ses-*")):
        session_id = os.path.basename(session_dir)
        fmap_dir = os.path.join(session_dir, "fmap")
        func_dir = os.path.join(session_dir, "func")

        reverse_epi_pattern = f"{subject_id}_{session_id}_task-REST_acq-1_dir-AP_epi.nii.gz"
        reverse_json_pattern = f"{subject_id}_{session_id}_task-REST_acq-1_dir-AP_epi.json"
        forward_epi_pattern = f"{subject_id}_{session_id}_task-REST_acq-1_dir-PA_epi.nii.gz"
        forward_json_pattern = f"{subject_id}_{session_id}_task-REST_acq-1_dir-PA_epi.json"

        reverse_epi_files = glob.glob(os.path.join(fmap_dir, reverse_epi_pattern))
        reverse_json_files = glob.glob(os.path.join(fmap_dir, reverse_json_pattern))
        forward_epi_files = glob.glob(os.path.join(fmap_dir, forward_epi_pattern))
        forward_json_files = glob.glob(os.path.join(fmap_dir, forward_json_pattern))

        if len(reverse_epi_files) == 1 and len(reverse_json_files) == 1 and len(forward_epi_files) == 0 and len(forward_json_files) == 0:
            # Find the first REST BOLD file in the func folder
            rest_bold_pattern = f"{subject_id}_{session_id}_task-REST_run-1_bold.nii.gz"
            rest_bold_files = glob.glob(os.path.join(func_dir, rest_bold_pattern))

            if len(rest_bold_files) > 0:
                # Load the REST BOLD NIfTI file
                rest_bold_file = rest_bold_files[0]
                rest_bold_img = nib.load(rest_bold_file)

                # Compute the mean image across the time dimension
                mean_img_data = np.mean(rest_bold_img.get_fdata(), axis=-1)
                mean_img = nib.Nifti1Image(mean_img_data, rest_bold_img.affine)

                # Save the mean image as the forward spin echo NIfTI file in the fmap folder
                forward_epi_file = os.path.join(fmap_dir, forward_epi_pattern)
                nib.save(mean_img, forward_epi_file)

                # Copy the JSON from the reverse spin echo to the forward spin echo
                reverse_json_file = reverse_json_files[0]
                forward_json_file = os.path.join(fmap_dir, forward_json_pattern)

                with open(reverse_json_file, "r") as src:
                    json_data = json.load(src)

                with open(forward_json_file, "w") as dst:
                    json.dump(json_data, dst, indent=4)

In [2]:
#import os
#import glob
#import json

# Define the base BIDS directory
#bids_dir = "/Users/bene.ramirez/projects/InfantDevelopment/NKIdev/data/developmental_in"

# Loop through all subjects
for subject_dir in glob.glob(os.path.join(bids_dir, "sub-*")):
    subject_id = os.path.basename(subject_dir)

    # Loop through all subject sessions
    for session_dir in glob.glob(os.path.join(subject_dir, "ses-*")):
        session_id = os.path.basename(session_dir)

        # Define the fmap and func directories
        fmap_dir = os.path.join(session_dir, "fmap")
        func_dir = os.path.join(session_dir, "func")

        # Loop through all JSON files in the fmap directory
        for json_file in glob.glob(os.path.join(fmap_dir, "*.json")):
            # Read the existing JSON content
            with open(json_file, "r") as f:
                json_data = json.load(f)

            # Add the new key "IntendedFor" if it does not exist
            if "IntendedFor" not in json_data:
                json_data["IntendedFor"] = []

            # Loop through all BOLD files in the func directory
            for bold_file in glob.glob(os.path.join(func_dir, f"{subject_id}_{session_id}_*_bold.nii.gz")):
                # Get the relative path of the BOLD file from the fmap directory
                relative_path = os.path.relpath(bold_file, fmap_dir)

                # Add the relative path of the BOLD file to the "IntendedFor" list if it's not already there
                if relative_path not in json_data["IntendedFor"]:
                    json_data["IntendedFor"].append(relative_path)

            # Write the updated JSON content back to the file
            with open(json_file, "w") as f:
                json.dump(json_data, f, indent=4)
